<a href="https://colab.research.google.com/github/alatieh/Claimed-Greening/blob/backup/claimed_Greening_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

The biosphere is the most significant and dynamic natural sink and emission of Carbon. The Biosphere "Greening" is often used to describe the increase in vegetation cover and productivity observed in many regions of the world in the latest 50 years. NDVI studies confirmed a general greening trend in terrestrial vegetation. Some studies theorize this phenomenon to the effects of rising CO2 levels in improving plant water use efficiency. The rising CO2 effect on vegetation is known as "CO2 fertilization". In comparison, others point to the role of direct human management of forests and agriculture, in addition to fertilization with nitrogen. However, this Greening is hard to explain and attribute proportionally to the corresponding factor.

The problem is that the term "Greening" may be misleading, as it does not reflect the reality of the ongoing anthropogenic degradation, deforestation, biodiversity loss, disruption, acidification, and pollution of the Earth's ecosystems. 

CO2 is a key indicator of the biosphere's photosynthesis and respiration, which reflects the health and activity of all living systems on Earth, including the oceans. Unlike NDVI, which measures only the surface reflectance of vegetation in the red and near-infrared bands, CO2 can provide a more comprehensive and reliable signal of the biosphere's vitality. In this research, I challenge the "Greening" claim using CO2 measurements from various locations worldwide. I will begin with the latest state of the art using the Global Carbon Project GCP report then I will compare my results and analysis to discuss the implications for our understanding of global change.

#Global Carbon Project GCP

The data from the Global carbon Project GCP [(Friedlingstein et al., 2022)](https://essd.copernicus.org/articles/14/4811/2022/)

In [ ]:
url = "https://data.icos-cp.eu/licence_accept?ids=%5B%221umMtgeUlhS2Y1YW_Qp94bu3%22%5D"  
filename = "Globalcarbon.xlsx"  

response = requests.get(url)

if response.status_code == 200:
    with open(filename, "wb") as f:
        f.write(response.content)
    excel_file = pd.ExcelFile(filename)
    sheet_name = "Global Carbon Budget"

    df1 = pd.read_excel(filename, sheet_name='Historical Budget', 
                        skiprows=range(15),usecols='A,C:H',parse_dates=[0],index_col=0)
    df1 = df1.iloc[-172:]
    df1.columns = ['land_use_change_emission', 'atmospheric_growth',
       'ocean_sink', 'land_sink', 'cement_carbonation_sink',
       'budget_imbalance']
    df1.fillna(value=0, inplace=True)
    # Check point to nullify the fluxes in df1 only
    #df1 = df1*0  


    df2 = pd.read_excel(filename, sheet_name='Fossil Emissions by Category', 
                        skiprows=range(8),usecols='A,C:H',parse_dates=[0],index_col=0)
    df2.columns = ['coal_emission',
       'oil_emission', 'gas_emission', 'cement_emission', 'flaring_emission',
       'other_emission']
    df2.fillna(value=0, inplace=True)
    df2 = df2 /1000  # convert from MtC to GtC

    # Check point to nullify the fossil fuel only
    #df2 = df2*0  

    #print(df1.head(),df2.head(5)) 
else:
    print("Failed to download", url)

GCP = pd.concat([df1,df2], axis=1)
time= GCP.index

#len(df)
#df.plot();plt.show()

In [ ]:
print("Absorbed Carbon in the ocean", round(GCP.ocean_sink.sum()), "GtC in total, or half kilo Carbon in every square meter = 1.8 kilo of CO2 in every square meter")

What is the density of Carbonic acid?   
1 101 kg/m3  according to https://en.wikipedia.org/wiki/Carbon_dioxide  
2 540 kg/m³  according to https://www.aqua-calc.com/page/density-table/substance/carbonic-blank-acid  
1 668 kg/m³  according to https://www.lifepersona.com/carbonic-acid-h2co3-properties-uses-and-significance  

so adding Carbon dioxide make the water denser then the volume will shrink?

### Emissions uncertainties of [Friedlingstein et al. (2022)](https://essd.copernicus.org/articles/14/4811/2022/)  
Emissions from **fossil fuel** combustion and industrial processes (uncertainty of **±5%** for a ± 1 sigma confidence level)  

Emissions from **land-use change** (uncertainty of **±0.7 GtC/yr**). Average of three bookkeeping models: H&N (Houghton &Nassikas, 2017), BLUE (Hansis, et al., 2015) and OSCAR (Gasser et al., 2020).  
The **land sink** (uncertainty of **±0.5 GtC/yr** on average) was estimated from the average of 16 dynamic global vegetation models.  

The **ocean sink** (uncertainty of **±0.4 GtC/yr** on average) is estimated from the average of 10 global ocean biogeochemistry models and the average of 7 ocean fCO2 data products.  
The budget imbalance is the sum of emissions (fossil fuel and industry + land-use change) minus (atmospheric growth + ocean sink + land sink + cement carbonation sink); it is a measure of our imperfect data and understanding of the contemporary carbon cycle.  

##plot data

In [ ]:
# check what we have imported
# --------------Visual inspection--------------# 
fig, (ax1, ax2) = plt.subplots(1, 2)

#plotting the CO2 emission's  & sinks fluxes
# emissions
plt.subplot(1,2,1)
GCP.coal_emission.plot(figsize=(22, 10),color='black',linestyle='-')
GCP.oil_emission.plot(color='dimgrey',linestyle=':')
GCP.gas_emission.plot(color='lightgrey',linestyle='-.')
GCP.flaring_emission.plot(color='black',linestyle=':')
GCP.cement_emission.plot(color='peru', linestyle=':')
GCP.other_emission.plot()
GCP.land_use_change_emission.plot( color='r')
# sinks
(GCP.land_sink*-1).plot(color='g')
(GCP.ocean_sink*-1).plot(color='b')
(GCP.cement_carbonation_sink*-1).plot(color='peru')

plt.legend()
plt.xlabel("year")
plt.ylabel("GtC/year")
plt.title("Annual CO2 emission's  & sinks fluxes")

#plotting the atmospheric growth and uncertainty
GCP.atmospheric_growth.plot(figsize=(22, 10),ax=ax2)
GCP.budget_imbalance.plot(ax=ax2)
plt.legend()
ax2.set_title("Annual Atmospheric CO2 growth and uncertainty")
ax2.legend()
fig.tight_layout()
plt.show()

## BoxPlot

In [ ]:
# --------------Statistical inspection--------------# 
#df.describe()
plt.figure(figsize=(22,8))
boxplot3= sns.boxplot(data=GCP.iloc[152:,:])
boxplot3.set(xlabel='Fluxes in GtC/year',ylabel='GtC/year')
boxplot3.axes.set_title("Statistical of the C fluxes in the last 20 years according to Global Carbon Budget 2022 ", fontsize=16)
boxplot3.tick_params(axis='x', labelrotation=270)
boxplot3.set_xticklabels(['land-use change','atm growth','ocean sink','land sink','cement sink','imbalance','coal','oil','gas','cement emisson','flaring','other'], fontsize=12)
boxplot3.axhline(0, linestyle="dashed")
plt.show()

### Visualize the statistical data of the Carbon budget

Here : https://youtu.be/RHU5A3IfDT4  

The plot begins with the whole data set from 1850 till 2021; then, it iterates to reproduce a boxplot for a shorter period. It ends with the fluxes in the last 21 years. The high variability in land Carbon sinks is apparent. The land sink grows faster than the ocean sink, and land-use changes emissions. In the next code line is the recipe for that visualization on YouTube.

## check the percentage of change

We define the percentage of change as:   

$(\frac{x_i}{x_{i-1}} -1)*100$   

where ${x_i}$ is the current value  

and $x_{i-1}$ is the previous value.

In [ ]:
# Calculate pct change 
tot_bio_sink_pct = (GCP.land_sink.sub(GCP.land_use_change_emission)).pct_change().dropna().mul(100)
tot_bio_sink_pct.plot(title='Total biosphere C sink (sink - emission) percentage of change', ylabel='Annual percentage cahnge %')
plt.show()

ocean_sink_pct = GCP.ocean_sink.pct_change().dropna().mul(100)
ocean_sink_pct.plot(title='Ocean C sink percentage of change', ylabel='Annual percentage cahnge %')
plt.show()

massive emission can happen due to wildfire but massive sink can not happen on land because terrestrial plants don’t increase exponentially in a short time. Should we think about the role of algal blooms in carbon sinks?

In [ ]:
print("The mean percentage of change of total land (sink - emission) is ",round(tot_bio_sink_pct.mean()),'% yearly')

In [ ]:
r, p = stats.pearsonr(tot_bio_sink_pct, ocean_sink_pct)
print("correlations value fro R is", round(r,2), 'And for P is',round(p,2))

I see my reasoning is incorrect because I separated the sink from the natural disequilibrium sink :(

## $\color{blue}{\text{Reject unrealistic changes of land sink, should I?}}$

In [ ]:
tot_bio_sink_pct['1973-01-01'] =np.nan
tot_bio_sink_pct['1999-01-01'] =np.nan
tot_bio_sink_pct['1924-01-01'] =np.nan


tot_bio_sink_pct.plot(title=' \'A bit realistic\' total biosphere C sink (sink - emission) percentage of change', ylabel='Annual percentage cahnge %')
plt.show()

In [ ]:
print("The mean percentage of change of total land (sink - emission) become NOW after deleting the unrealistic values",round(tot_bio_sink_pct.mean()),'% yearly')

In [ ]:
#check the hist and if there is any correlation
sns.jointplot(x=tot_bio_sink_pct, y=ocean_sink_pct, kind="reg")

## Closing the budget  

In [ ]:
print("The total absolute fluctuations sum of the Budget Imbalance is", round((np.abs(GCP.budget_imbalance)).sum(),2) ,
      "GtC for all the periods from 1850 to 2022. ","\n"
      "The total sum of the Budget Imbalance is", round(GCP.budget_imbalance.sum(),2), "GtC","\n"
      "This extra 13.11 GtC sink affirms the fact of missing sinks from the budget, like the biological pump")

The land sink is calculated initially in the GCP as a residual. I added the budget imbalance to the land sink to close the budget

In [ ]:
residual= GCP.land_sink + GCP.budget_imbalance

residual.plot(label="total of land sink and budget imbalance''residual'")
GCP.land_sink.plot(label='land sink')
GCP.budget_imbalance.plot(label='imbalance')
plt.legend()


## Biological Carbon Pump **BCP** and other fluxes that end to ocean


 The quantification of Biological Carbon Pump BCP varies widely. The variation in estimation begins with the fluctuations of primary productivity in the Euphotic zone. [Behrenfeld et al. (2006)](https://www.nature.com/articles/nature05317) show a ~4% variation in surface chlorophyll using Remote sensing and the SeaWiFS instrument. The exported C from the sunlit zone to the ocean interior is ~10 GtC/year ([Earth Observatory website](https://earthobservatory.nasa.gov/features/Phytoplankton)). Finally, ~2 GtC/year will be deposited on the ocean floor, where it will degrade. The final **buried** C is estimated to be between **0.2 and 0.4** GtC a year ([Middelburg, 2019](https://link.springer.com/book/10.1007/978-3-030-10822-9)). In the IPCC report that 0.2 GtC is buried ([Canadell et al.,2021](https://www.ipcc.ch/report/ar6/wg1/downloads/report/IPCC_AR6_WGI_Chapter05.pdf), p700). The study by [Buesseler et al. (2020)](https://doi.org/10.1073/pnas.1918114117) thinks that BCP is twice more efficient in carbon sequestration than we thought before.   
 
The effect of climate change on BCP has yet to be fully understood. The feedbacks on BCP are both positive and negative ([Canadell et al.,2021](https://www.ipcc.ch/report/ar6/wg1/downloads/report/IPCC_AR6_WGI_Chapter05.pdf),  p682). Higher CO2 enhances phytoplankton growth rate ([WOLF-GLADROW et al., 2002](https://doi.org/10.1034/j.1600-0889.1999.00023.x)).   

In this simple model, I will merge the terrestrial rivers' carbon input with the BCP Because both deliver carbon to the ocean. The exported carbon from rivers to oceans varies between **0.35** GtC/year (Raymond & Spencer, 2015) and **0.8** by (Canadell et al.,2021, p700). The rivers signal will affect the model because its δ13C values which vary between −21 to −29‰ ([Qi et al. 2020](https://www.nature.com/articles/s41467-020-18808-8 ))    

Finally, I also considered the marine rain carbon flux. Marine rain washes the dust and dissolves the carbon dioxide from the atmosphere. [Willey et al. (2000)](https://doi.org/10.1029/1999GB900036 ) calculated almost ~0.5 GtC/year of carbon delivered by rain to the surface ocean as DOC or DIC.I will put the minimum expectation at zero GtC/year because this flux is delivered to the ocean's surface; it is vulnerable to degradation and reemitting back to the atmosphere.  

Flux | minimum magnitude GtC/year| Maximum magnitude GtC/year
----- | --------| ------
Biological pump |0.2 | 0.8
Rivers' carbon |0.35 | 0.8
 Rain |0 | 0.5
  |-------------- |--------------
Total |0.55|  2.1

GCP considers all residuals as a Land Sink. However, I will take part of this residual flux and assign it to the ocean biological pump. So the assigned portion of carbon will flow into the ocean surface instead of the biosphere. The factor will be between 0 and 1 of the GCP Land_sink flux. I will call this factor "biological_pump_percentage." In the following code line, I will plot it to see if it is near the expected flux magnitude.

In [ ]:
biological_pump_percentage = 0  # instead of 0.2
# However, I will take part of this residual flux and assign it to the ocean biological pump

# biological_pump = residual* biological_pump_percentage
Land_sink_nettto = residual * (1- biological_pump_percentage)
# biological_pump.plot(label='biological pump according to this model', color='black')

## Mummified Carbon flux
After that, the "Land_sink_nettto" flux ends up in land. Part of this carbon is mummified for a very long time.  
`Here I suggest a new principle of "Carbon mummification".` Carbon is mummified due to pollutions, contamination, anoxia, or becoming chemical compounds with very long degradation time like charcol, or rapid environmental change that kill the organisms massively, or other Carbon mummification process.  

Waste dumping on land or ocean and plastic packaging that prevents carbon from escaping are examples of the Carbon mummification process. The waste dumping practice considered as part of this Carbon Mummified reservoir because Carbon will not have enough time to fully decay due to high accumulation of waste-dump or even from algal bloom.  

This flux will be subtracted from the **biosphere reservoir**. The magnitude of the mummified_C_flux will also be **as** fraction of the Land_sink_nettto   
It's **not** subtracted from the land sink, it's proportional to the land sink.  
In the subsequent code lines, I will code the percentage of this Mummified Carbon flux, plot it, and then explain and try to quantify its contents fluxes, namely the:  
1. Black Carbon


#### **1. Black Carbon**   
Black carbon BC, also called Carbonaceous particles or soot or pyrogenic carbon, are mixtures  of fine particles of the incomplete combustion of fossil fuel. BC floats in the air after combustion then deposits or washed onto the land or ocean. Rainwater washes the BC and other organic compounds to the surface water and ocean. Black Carbon BC mainly originated from wildfires and burning biomass.BC has residence time of 5,387 years ([Bowring et al. 2022](https://www.nature.com/articles/s41561-021-00892-0)), hence, BC will disappear from Carbon Cycle in short term.

The magnitude of the flux has high uncertainty:  
* [Jones et al. (2019)]( https://www.nature.com/articles/s41561-019-0403-x) reported almost a ~third Giga ton of Black Carbon a year. They estimated 60 GtC accumulated since 1750 from landscape fires. In their words “Our central estimate equates to **12% of the annual carbon emitted globally by landscape fires.  
~33–40% of the global biomass carbon (60 Pg) lost through land use change  since 1750**”.  
* whereas [Xu et al., (2011)](https://doi.org/10.1021/acs.est.1c03117) reported just 0.01 GtC/year.Thirty times smaller than the previous estimation of Jones et al. (2019).  
* [Qi et al. (2020)](https://www.nature.com/articles/s41467-020-18808-8 ) estimated the BC from biomass burning ~(0.114–0.383 GtC/year) but they estimated very small flux of BC from fossil fuel combustion (0.002–0.029 GtC/year)   
* ([Bowring et al. 2022](https://www.nature.com/articles/s41561-021-00892-0)) calculation was an average of 0.34 GtC/year from Wildfires in the periode from 1901 to 2010.   


BC and other pollutions can affect our model because it takes the depleted d13C atoms from the box-model so I need to test it. beside the isotopic balance issue. The BC and burned charcol can be stored in ocean and soil for for centuries to millennia ([Qi et al. 2020](https://www.nature.com/articles/s41467-020-18808-8)). Because these long turning over time, BC disappears from the active system. [Jones et al. (2019)]( https://www.nature.com/articles/s41561-019-0403-x) pointed out that Global Carbon Project GCP misses the Pyrogenic carbon flux which results in calculation errors in the budget. 

#### **Other Carbon fluxes quantification**  

1. Black carbon from Incomplete burning (Even the wildfire has this). so it has two sources (fossil fuels and land use change).  

2. washing of organic matter from atmosphere to the ocean and land
3. Urban Sewage and waste   
4. Agriculture waste, food waste, Woodworking, ash, animal waste (liquid and solid)  
5. Enhancement of food storage with freezing, drying, oxygen free packaging, salting, and silos.    
6. Chemicals effect on decay time (pesticides, herbicides, fungicides,antibiotics, and plastic) and expanding of marine anoxia zones. These effect mostly the bacteria & archaea reservoir but also the vegetation(≈77 Gt C)
7. River organic matter flux to the ocean  
8. Weathering flux  
9. inert biological polymers produced by plants and animals like Sporopollenin of pllen

## Volcanic Carbon
The current carbon emission estimation from the volcanic eruption has very few measurements because the measurements of volcanic plumes are challenging, with a tiny amount of many carbon species inside the emitted water vapor (Burton et al., 2013). Besides volcanism as a source of Carbon, other sources play an important role, like the non-volcanic inorganic degassing from tectonically active areas and mud volcanism (Burton et al., 2013). The d13C of volcanism Carbon depends on the source; Mid Ocean Ridge Basalt MORB volcanism is -6.5‰, whereas subducted limestone is almost ~0‰ ([Fischer & Lopez,  2016](https://doi.org/10.1002/2016GL068499)). For example, Fischer & Lopez., ([2016](https://doi.org/10.1002/2016GL068499)) measurement of a volcanic eruption in the Western Aleutians was -4.4‰, which indicates the MORB origin. Whereas, Rizzo et al. ([2015](https://doi.org/10.1016/j.chemgeo.2015.07.007)) reported a range of values for Mount Etna from − 1.3‰ to + 1.5‰. However, in these different ranges, [Sano &  Marty (1995)](https://doi.org/10.1016/0009-2541(94)00097-R) reported that most of the volcanic CO2 is produced by the "decarbonization of subducted marine limestone," which makes volcanic Carbon with a higher d13C signature. In this model, I will assume a value of **0.8‰** for all volcanic Carbon.In this way, I assumed that volcanic Carbon is 80% from lime origin (0‰), and only 20% from MORB origin with (-4‰)  
I will use a value of **0.25 GtC/year** in this model for subaerial volcanic Carbon as in (Burton et al. [2013](https://pubs.geoscienceworld.org/msa/rimg/article-abstract/75/1/323/140959))

### weathering
Burton et al., (2013) repoted 0.14 Gt/yr due to silicate weathering

In [ ]:
# weathering = np.repeat(0.25, 172)

### Petrochemical
In the following lines, I will estimate the Petrochemical fluxes and then try to check if they can affect the model.   
  
Not all the reported fossil fuel is burned into the atmosphere. Many products are made from fossil fuel but end up not-burned in other places.  Take for example the leakages during transfer and usage of a car diesel leak or ship tanker washing. Most importants are the Petrochemical Feedstocks. Fossil fuels are incorporated **in** many Petrochemical Feedstocks like fertilizer, rubber, clothes, fabrics, asphalt, and various plastics. Most petrochemicals like plastics or asphalt remain in the environment and are not burnt. [IEA (2018)](https://www.iea.org/reports/the-future-of-petrochemicals) estimates that **14% of oil** and **8% of gas** used in Petrochemical Feedstocks. Removing non-combusted fossil fuel from the budget will affect mainly the oil becasue it's the most used in petrochemical industry ([Andrew, 2020,](https://doi.org/10.5194/essd-12-1437-2020) P6).    


GCP fossil fuel dataset based on the Carbon Dioxide Information Analysis Center CDIAC. Emission factors are used to calculate the emissions and discriminate the non-combusted fossil. Emission factors are applied to coal and natural gas ([Andrew, 2020](https://doi.org/10.5194/essd-12-1437-2020)). "*The fossil CO2 dataset accounts for some share of carbon that is not oxidized. Nevertheless, GCP does not calculate explicitly how much of this is black carbon or petrochemical*" (personal communication with Dr. Robbie Andrew, 2023).  
Because "*True*  emissions  cannot  be  known" according to Andrew ([2020](https://doi.org/10.5194/essd-12-1437-2020)), In the following code line, I test to subtract [IEA](https://www.iea.org/reports/the-future-of-petrochemicals) estimation of Petrochemical usage of oil and gas. This test its possibility influence on the model.    

##The correlation between the total C sink and the difference between North and south pole measurements

In [ ]:
# plot total sink and the difference between North and south pole measurements
total_C_sink = GCP.land_sink+ GCP.ocean_sink
North_min_south_pole_c_pool= ptb_CO2.ptb_CO2_without_Seasonality-south_pole_CO2.south_pole_CO2_without_Seasonality

plt.plot(total_C_sink, label="Total sink of biosphere & ocean")
plt.plot(North_min_south_pole_c_pool, label='Difference between North and south pole measurements')
plt.title("The correlation between the total C sink and the difference between North and south pole measurements")
plt.ylabel('GtC')

plt.legend()
plt.show()

In [ ]:
# zoom in
plt.plot(total_C_sink['1961-01-01':], label="Total sink of biosphere & ocean")
plt.plot(North_min_south_pole_c_pool, label='Difference between North and south pole measurements')
plt.title("The correlation between the total C sink and the difference between North and south pole measurements")
plt.ylabel('GtC')

plt.legend()
plt.show()

#References 

<br> **